In [1]:
from __future__ import print_function
import requests
import numpy as np

In [2]:
def get_ans():
	'''Asks user for input. If the user types yes true is returned'''
	ans = input()
	if ans.lower == 'y' or ans.lower == 'yes' or ans == '1':
		return True
	else:
		return False

def get_wcif(comp):
	'''Pulls WCIF from WCA and return json object. If there is a problem with the request the program terminates'''
	print('Attempting to pull competition info...')
	# Send GET request for WCIF
	WCIF = requests.get('https://www.worldcubeassociation.org/api/v0/competitions/' + comp + '/wcif/public')
	# If response code is 200 everything good, otherwise something bad
	if WCIF.status_code == 200:
		print('Success!')
		return WCIF.json()
	else:
		print('Response Error!')
		exit()

In [3]:
# Get comp name from user
print('Enter competition ID:')
#comp = input()
comp = 'Cubinginthe6ix2019'
#comp_file = comp + '-registration.csv'

WCIF = get_wcif(comp)

Enter competition ID:
Attempting to pull competition info...
Success!


In [95]:
class Competitor:
    def __init__(self, name, ID, events, wcaid):
        '''Competitor constructor'''
        self.name = name
        self.id = ID
        self.events = events
        self.wcaid = wcaid
        
    @staticmethod
    def build_competitors(WCIF):
        '''Builds an array of competitors from WCIF data'''
        lst = [None] * len(WCIF['persons']) # Allocate memory
        i = 0 # Counter
        for pers in WCIF['persons']: # Loops over each person registered
            # Build competitors
            lst[i] = Competitor(pers['name'], pers['registrantId'], pers['registration']['eventIds'], pers['wcaId'])
            if lst[i].wcaid == None:
                lst[i].wcaid = ''
            i += 1
        return lst
    
    def write_nametags(comp, f_name, num_blank = 0):
        '''Given an array of competitors, code for nametags is written for LaTeX'''
        f = open(f_name, 'w')
        for i in range(len(comp)):
            f.write('\\nametag{%s}{COMPETITOR}{%s}%%\n' % (comp[i].name, comp[i].wcaid))
        for i in range(num_blank): # Include blank nametags for day of registrations
            f.write('\\nametag{}{COMPETITOR}{}%\n')
        f.close()
        
        
class Competition:
    @staticmethod
    def centi2min(centi):
        '''Converts a time in centiseconds to minute:second format'''
        minute = int(np.floor(centi / 6000)) # Compute number of minutes
        sec = int(np.ceil((centi - 6000 * minute) / 100)) # Compute remainder
        return str(minute) + ':' + str(sec).zfill(2) # Format string (zfill zero pads)
    
    def build_events(WCIF):
        '''Build a list of events held at the competition'''
        lst = [None] * len(WCIF['events'])
        for i in range(len(WCIF['events'])):
            lst[i] = WCIF['events'][i]['id']
        return lst
    
    def build_cutoffs(WCIF):
        '''Build a list of cutoffs for the competition. Note: Assumes cutoffs only in the first round'''
        lst = [None] * len(WCIF['events'])
        for i in range(len(WCIF['events'])):
            if WCIF['events'][i]['rounds'][0]['cutoff'] == None:
                lst[i] = ''
            else:
                lst[i] = Competition.centi2min(WCIF['events'][i]['rounds'][0]['cutoff']['attemptResult'])
        return lst
    
    def build_limits(WCIF):
        '''Build a list of time limits for the competition'''
        lst = [None] * len(WCIF['events'])
        for i in range(len(WCIF['events'])):
            if WCIF['events'][i]['rounds'][0]['cutoff'] == None:
                lst[i] = ''
            else:
                lst[i] = Competition.centi2min(WCIF['events'][i]['rounds'][0]['timeLimit']['centiseconds'])
        return lst
    
    def build_rounds(WCIF):
        '''Build a list of additional rounds for the competition'''
        lst = [[] for i in range(len(WCIF['events']))]
        for i in range(len(WCIF['events'])):
            if WCIF['events'][i]['rounds'][0]['advancementCondition'] != None:
                for j in range(len(WCIF['events'][i]['rounds']) - 1):
                    lst[i].append(WCIF['events'][i]['rounds'][j]['advancementCondition']['level'])
        return lst
    
    
    def __init__(self, WCIF):
        '''Competition constructor'''
        self.name = WCIF['name']
        self.events = Competition.build_events(WCIF)
        self.cutoffs = Competition.build_cutoffs(WCIF)
        self.limits = Competition.build_limits(WCIF)
        self.rounds = Competition.build_rounds(WCIF)
            

In [96]:
c = Competitor.build_competitors(WCIF)
comp = Competition(WCIF)

In [97]:
print(comp.cutoffs)
print(comp.limits)
print(comp.rounds)

['', '1:30', '4:00', '6:00', '']
['', '10:00', '6:00', '8:00', '']
[[40, 16], [16], [], [], []]


In [ ]:
def write_groups(df, tex_f = 'Groups.tex', wca_f = 'Groups.md', csv_f = 'Groups.csv'):
	# String array for LaTeX code and WCA website for groups
	tex_groups = np.array(r'\groups{' + df['Name'].map(str) + '}{')
	wca_groups = np.array(df['Name'].map(str) + ' |')
	wca_header = np.array(['Name |', ' --- |'])
	for i in list(df)[2:]:
		tex_groups += event_dict[i][0] + ' & ' + df[i].map(str) + ' \\\ '
		wca_groups += ' ' + df[i].map(str) + ' |'
		#wca_header += np.array([' ' + event_dict[i][1] + ' |', ' :---: |'])
		wca_header = np.core.defchararray.add(wca_header, [' ' + event_dict[i][1] + ' |', ' :---: |']) # There's got to be an easier way
	tex_groups += '}%' # Close the bracket in the string array
	# Write groups to files
	np.savetxt(tex_f, tex_groups, fmt = '%s')
	np.savetxt(wca_f, np.hstack((wca_header, wca_groups)), fmt = '%s')
	df.to_csv(csv_f, index = False)

# Create groups for one event
def event_group(df, g_size = 16):
	# df should be two columns, Name and an eventID
	df_old = df
	df = df[df.iloc[:, 1] == 1] # Extract only competitors in event
	df = df.sample(frac = 1) # Randomize groups
	n = len(df) # Number of competitors in event
	num_g = int(np.round(n / float(g_size))) # Number of groups
	if list(df)[1] in ['333fm', '333mbf', '444bf', '555bf']:
		# For long events, only 1 group
		group = np.ones(n).astype(int)
	else:
		group = np.arange(n) * num_g / n + 1
	df['Group'] = group.astype(str) # Append column formatted as a string
	return df_old.merge(df, how = 'left').fillna('') # Return merged groups with old DataFrame, filling blanks with an empty string

def make_groups(df):
	group_df = df[['index', 'Name']].copy()
	# Iterate through columns of registration file to calculate groups for each event
	for i in list(df)[7:-3]:
		g_num = event_group(df[['Name', i]])['Group'].map(str)
		group_df[i] = g_num
	return group_df